In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
# import pandas_profiling
import folium
from folium import plugins

In [2]:
# aisles = pd.read_csv("aisles.csv")
# departments = pd.read_csv("departments.csv")
order_products = pd.read_csv("order_products.csv")
orders = pd.read_csv("orders.csv")
# products = pd.read_csv("products.csv")

In [3]:
order_products

,order_id,product_id,add_to_cart_order
0,2,33120,1
1,2,28985,2
2,2,9327,3
3,2,45918,4
4,2,30035,5
...,...,...,...
33819101,3421063,14233,3
33819102,3421063,35548,4
33819103,3421070,35951,1
33819104,3421070,16953,2


In [4]:
orders

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,1,2,8,NaN
1,2398795,1,2,3,7,15.0
2,473747,1,3,3,12,21.0
3,2254736,1,4,4,7,29.0
4,431534,1,5,4,15,28.0
...,...,...,...,...,...,...
3421078,2266710,206209,10,5,18,29.0
3421079,1854736,206209,11,4,10,30.0
3421080,626363,206209,12,1,12,18.0
3421081,2977660,206209,13,1,12,7.0


In [5]:
#根据order_number,确定非prior
grouped = orders.groupby(["user_id"])["order_number"].max()
grouped_df=pd.DataFrame(grouped)
grouped_df["eval_set_1"]=0
grouped_df

,order_number,eval_set_1
user_id,,
1,11,0
2,15,0
3,13,0
4,6,0
5,5,0
...,...,...
206205,4,0
206206,68,0
206207,17,0


In [6]:
#填充prior
orders_2 = pd.merge(orders, grouped_df, how='left', on=['order_number','user_id'])
orders_2['eval_set_1'].fillna("Prior", inplace=True)
orders_2

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,eval_set_1
0,2539329,1,1,2,8,NaN,Prior
1,2398795,1,2,3,7,15.0,Prior
2,473747,1,3,3,12,21.0,Prior
3,2254736,1,4,4,7,29.0,Prior
4,431534,1,5,4,15,28.0,Prior
...,...,...,...,...,...,...,...
3421078,2266710,206209,10,5,18,29.0,Prior
3421079,1854736,206209,11,4,10,30.0,Prior
3421080,626363,206209,12,1,12,18.0,Prior
3421081,2977660,206209,13,1,12,7.0,Prior


In [7]:
#找到非prior的
orders_part=orders_2[orders_2["eval_set_1"]==0]
orders_part

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,eval_set_1
10,1187899,1,11,4,8,14.0,0.0
25,1492625,2,15,1,11,30.0,0.0
38,2774568,3,13,5,15,11.0,0.0
44,329954,4,6,3,12,30.0,0.0
49,2196797,5,5,0,11,6.0,0.0
...,...,...,...,...,...,...,...
3420933,1716008,206205,4,1,16,10.0,0.0
3421001,1043943,206206,68,0,20,0.0,0.0
3421018,2821651,206207,17,2,13,14.0,0.0
3421068,803273,206208,50,5,11,4.0,0.0


In [8]:
#确定order_id
order_products_part=order_products[["order_id"]].drop_duplicates()
order_products_part

,order_id
0,2
9,3
17,4
30,5
56,6
...,...
33819080,3421049
33819086,3421056
33819091,3421058
33819099,3421063


In [9]:
#填充train
order_products_part["eval_set_2"]="Train"
order_products_part

,order_id,eval_set_2
0,2,Train
9,3,Train
17,4,Train
30,5,Train
56,6,Train
...,...,...
33819080,3421049,Train
33819086,3421056,Train
33819091,3421058,Train
33819099,3421063,Train


In [10]:
#填充train
order_part_2=pd.merge(orders_part,order_products_part,how="left", on ='order_id')
order_part_2

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,eval_set_1,eval_set_2
0,1187899,1,11,4,8,14.0,0.0,Train
1,1492625,2,15,1,11,30.0,0.0,Train
2,2774568,3,13,5,15,11.0,0.0,NaN
3,329954,4,6,3,12,30.0,0.0,NaN
4,2196797,5,5,0,11,6.0,0.0,Train
...,...,...,...,...,...,...,...,...
206204,1716008,206205,4,1,16,10.0,0.0,Train
206205,1043943,206206,68,0,20,0.0,0.0,NaN
206206,2821651,206207,17,2,13,14.0,0.0,NaN
206207,803273,206208,50,5,11,4.0,0.0,NaN


In [11]:
#填充test
order_part_2['eval_set_2'].fillna("Test", inplace=True)
order_part_3=order_part_2.drop(['order_number','order_dow','order_hour_of_day','days_since_prior_order','eval_set_1'],axis=1)
order_part_3

,order_id,user_id,eval_set_2
0,1187899,1,Train
1,1492625,2,Train
2,2774568,3,Test
3,329954,4,Test
4,2196797,5,Train
...,...,...,...
206204,1716008,206205,Train
206205,1043943,206206,Test
206206,2821651,206207,Test
206207,803273,206208,Test


In [12]:
#两个eval_set放到一起
orders_3=pd.merge(orders_2,order_part_3,how="left", on =['order_id','user_id'])
orders_3

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,eval_set_1,eval_set_2
0,2539329,1,1,2,8,NaN,Prior,NaN
1,2398795,1,2,3,7,15.0,Prior,NaN
2,473747,1,3,3,12,21.0,Prior,NaN
3,2254736,1,4,4,7,29.0,Prior,NaN
4,431534,1,5,4,15,28.0,Prior,NaN
...,...,...,...,...,...,...,...,...
3421078,2266710,206209,10,5,18,29.0,Prior,NaN
3421079,1854736,206209,11,4,10,30.0,Prior,NaN
3421080,626363,206209,12,1,12,18.0,Prior,NaN
3421081,2977660,206209,13,1,12,7.0,Prior,NaN


In [13]:
#合并columns
orders_3['eval_set_2'].fillna("Prior", inplace=True)
orders_3['eval_set']=orders_3['eval_set_2']
orders_new=orders_3.drop(['eval_set_1','eval_set_2'],axis=1)
orders_new

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,eval_set
0,2539329,1,1,2,8,NaN,Prior
1,2398795,1,2,3,7,15.0,Prior
2,473747,1,3,3,12,21.0,Prior
3,2254736,1,4,4,7,29.0,Prior
4,431534,1,5,4,15,28.0,Prior
...,...,...,...,...,...,...,...
3421078,2266710,206209,10,5,18,29.0,Prior
3421079,1854736,206209,11,4,10,30.0,Prior
3421080,626363,206209,12,1,12,18.0,Prior
3421081,2977660,206209,13,1,12,7.0,Prior
